## Cell 0: preamble
Install required libraries.

In [ ]:
!pip install -q ccxt ccxtpro pandas pyarrow matplotlib

## Cell 1: imports

In [ ]:

import asyncio
import ccxt
import ccxt.pro as ccxtpro
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from pathlib import Path


## Cell 2: config loader

In [ ]:

import yaml
from dotenv import load_dotenv
import os

load_dotenv()

def load_config(path='config.yml'):
    if Path(path).exists():
        with open(path) as f:
            cfg = yaml.safe_load(f)
    else:
        cfg = {}
    cfg.update({
        'apiKey': os.getenv('API_KEY'),
        'secret': os.getenv('API_SECRET'),
        'symbol': cfg.get('symbol', 'BTC/USDT'),
        'Q_max': cfg.get('Q_max', 1.0),
    })
    masked_key = cfg['apiKey'][:4] + '...' if cfg['apiKey'] else None
    print('Loaded config for symbol', cfg['symbol'], 'API', masked_key)
    return cfg

config = load_config()


## Cell 3: exchange connect

In [ ]:

async def get_exchange(async_ws=False):
    if async_ws:
        exchange = ccxtpro.binance({'enableRateLimit': True})
    else:
        exchange = ccxt.binance({'enableRateLimit': True})
    await exchange.load_markets() if async_ws else exchange.load_markets()
    return exchange

# Instantiate REST and WS
rest = ccxt.binance({'enableRateLimit': True})
ws = None
try:
    rest.load_markets()
    print('REST connected')
except Exception as e:
    print('REST connection failed:', e)


## Cell 4: helper classes

In [ ]:

class Ingest:
    async def stream_orderbook(self, exchange, symbol, depth=20):
        while True:
            try:
                ob = await exchange.watch_order_book(symbol, depth)
                yield ob['timestamp'], ob['bids'], ob['asks']
            except Exception as e:
                print('WS error', e)
                await asyncio.sleep(1)

class Features:
    def update_sigma(self, prices):
        return np.std(np.diff(np.log(prices))) * np.sqrt(len(prices))

    def update_vpin(self, trades):
        return np.clip(np.random.rand(), 0, 1)  # placeholder

class Params:
    def update_k(self, slip_spread, tau):
        return 0.1 * tau

class Inventory:
    def __init__(self, qmax=1.0):
        self.q = 0.0
        self.qmax = qmax

    def update(self, dq):
        self.q = max(-self.qmax, min(self.qmax, self.q + dq))
        return self.q

class Quotes:
    def make_quotes(self, mid, k, tau, kappa, Q):
        delta_info = k * tau
        psi = kappa * Q
        ask = mid + delta_info - psi
        bid = mid - delta_info - psi
        return bid, ask

class Orders:
    def sync_quotes(self, bid, ask):
        print(f'Would place bid {bid} ask {ask}')

class Metrics:
    def calc_metrics(self, fills, inventory):
        return {'pnl': np.sum(fills)}

class Plotting:
    def plot_pnl(self, df):
        df.cumsum().plot()
        plt.show()

# unit tests

def run_unit_tests():
    q = Quotes()
    bid, ask = q.make_quotes(100, 0.1, 0.5, 0.01, 0)
    assert ask >= bid >= 0
    inv = Inventory()
    inv.update(1)
    assert inv.q == 1
    print('All unit tests passed')


## Cell 5: historical back-fill

In [ ]:

import pandas as pd
try:
    trades = rest.fetch_trades(config['symbol'], limit=50)
    prices = [t['price'] for t in trades]
except Exception as e:
    print('Fetch trades failed:', e)
    prices = [30000 + i for i in range(50)]

features = Features()
initial_sigma = features.update_sigma(prices)
print('Initial sigma', initial_sigma)


## Cell 6: live async loop

In [ ]:

async def live_loop():
    global ws
    ws = await get_exchange(async_ws=True)
    ingest = Ingest()
    params = Params()
    quotes = Quotes()
    inventory = Inventory(config['Q_max'])
    kappa = 0.01
    async for ts, bids, asks in ingest.stream_orderbook(ws, config['symbol']):
        mid = (bids[0][0] + asks[0][0]) / 2
        tau = features.update_vpin([])
        k = params.update_k(asks[0][0]-bids[0][0], tau)
        bid, ask = quotes.make_quotes(mid, k, tau, kappa, inventory.q)
        Orders().sync_quotes(bid, ask)
        inventory.update(0)
        break

await live_loop()


## Cell 7: metrics snapshot

In [ ]:

metrics = Metrics()
print(metrics.calc_metrics([0.1, -0.05], 0))


## Cell 8: shutdown handler

In [ ]:

if ws:
    await ws.close()
print('Shutdown complete')


## Cell 9: plots

In [ ]:

plotter = Plotting()
plotter.plot_pnl(pd.Series([0.1, -0.05, 0.2]))


## Cell 10: unit tests

In [ ]:

run_unit_tests()


## Cell 11: backtest demo

In [ ]:

print('Backtest demo placeholder')


## Cell 12: FAQ / next steps
TODO: add futures hedge, live OMS wrapper, dockerisation.